In [ ]:
import pandas as pd
import os
import numpy as np
from tqdm import tqdm
import math
import random
from sklearn.metrics import f1_score, roc_auc_score, confusion_matrix
import time
import matplotlib.pyplot as plt
import networkx as nx
from tqdm import tqdm
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph
import dgl.function as fn
from dgl.nn.pytorch import GraphConv, SAGEConv

In [ ]:
from deepsurv_utils import c_index, adjust_learning_rate
from loss import NegativeLogLikelihood

In [ ]:
# load data
all_patient_info = pd.read_csv("v")
stage1 = list(np.load("labels/name_stage1.npy"))
stage2 = list(np.load("labels/name_stage2.npy"))
patint_list = [*stage1, *stage2]
patient_info = all_patient_info[all_patient_info['folder_name'].isin(patint_list)]
feature_files = os.listdir("trans_feature")

In [ ]:
data = []
name = []
for feature_name in feature_files:
    path = "trans_feature/"+feature_name
    name.append(int(feature_name[:-4]))
    feature = list(np.load(path, allow_pickle=True))
    data.append(feature)
feature_data = pd.DataFrame(data)
feature_data['folder_name']=name
all_data = patient_info.merge(feature_data, how='left', on='folder_name')

In [ ]:
train_id = np.load("data_ind/train_index.npy",allow_pickle=True)
val_id = np.load("data_ind/val_index.npy", allow_pickle=True)
test_id = np.load("data_ind/test_index.npy",allow_pickle=True)
idx_train = torch.LongTensor(train_id)
idx_val = torch.LongTensor(val_id)
idx_test = torch.LongTensor(test_id)

print("training survival distribution:")
print(all_data.iloc[train_id,:]['OS_Status'].value_counts())
print("validation survival distribution:")
print(all_data.iloc[val_id,:]['OS_Status'].value_counts())
print("test survival distribution:")
print(all_data.iloc[test_id,:]['OS_Status'].value_counts())


# print("training survival distribution:")
# print(all_data.iloc[train_id,:]['RFS_Status'].value_counts())
# print("validation survival distribution:")
# print(all_data.iloc[val_id,:]['RFS_Status'].value_counts())
# print("test survival distribution:")
# print(all_data.iloc[test_id,:]['RFS_Status'].value_counts())

In [ ]:
import scipy.stats as st
data = all_data['RFS_Status'].to_list()
print(st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data)) )

In [ ]:
# define similarity of two patient
def SimScore(a1,a2,s1,s2,l1,l2,h1,h2,t1,t2,n1,n2,m1,m2,tnm1,tnm2): 
    c_score = 0
    h_score = 0
    t_score = 0
    # sex and age
    if s1 == s2:
        c_score +=1
    if abs(a1-a2) <= 5:
        c_score +=1
    
    if l1 == l2:
        h_score +=1
    if h1 == h2:
        h_score +=1
    
    if t1 == t2:
        t_score +=1
    if n1 == n2:
        t_score +=1
    if m1 == m2:
        t_score +=1
#     if tnm1 == tnm2:
#         t_score +=1

    return c_score*t_score*h_score

# def SimScore(a1,a2,s1,s2,l1,l2,h1,h2,t1,t2,n1,n2,m1,m2,tnm1,tnm2): 

#     return c_score*t_score*h_score


def adj_matrix(patient_info):
    age = patient_info['Age'].to_list()
    sex = patient_info['Sex_1_male_2_female'].to_list()
    
    loc = patient_info['Location_1_LUL_2_LLL_3_RUL_4_RML_5_RLL'].to_list()
    his = patient_info['Histology_1_Adenocarcinoma_2_SquamousCellCarcinoma_3_Others'].to_list()
    pts = patient_info['pT_Stage'].to_list()
    pns = patient_info['pN_Stage'].to_list()
    pms = patient_info['pM_Stage'].to_list()
    tnm = patient_info['pTNM'].to_list()

    edge_list=[]
    edge_wight=[]
    n_sample = len(age)
    adj = np.zeros((n_sample, n_sample))
    for i in tqdm(range(n_sample)):
        for j in range(n_sample):
            adj[i,j] = SimScore(age[i],age[j],sex[i],sex[j],loc[i],loc[j],his[i],his[j],
                                pts[i],pts[j],pns[i],pns[j], pms[i],pms[j],tnm[i],tnm[j])
            if adj[i,j] != 0:
                edge_list.append([i,j])
                edge_wight.append(adj[i,j])
    return adj, edge_list,edge_wight

In [ ]:
adj, edge_list, edge_wight = adj_matrix(all_data)
print("the number of edges in this graph:",len(edge_list))
print("Number of average degree: ",len(edge_list)/1705 )

In [ ]:
# save the labels
norm_label = all_data['OS_Month']
# norm_label = (all_data['OS_Month']-np.min(all_data['OS_Month']))/(np.max(all_data['OS_Month'])-np.min(all_data['OS_Month']))
labels = torch.from_numpy(norm_label.to_numpy())
events = torch.from_numpy(all_data['OS_Status'].to_numpy())
# build graph struture data
g = dgl.DGLGraph()
g.add_nodes(len(labels))
# add nodes
# node_feature = (all_data.iloc[:, 15:]-all_data.iloc[:, 15:].min())/(all_data.iloc[:, 15:].max()- all_data.iloc[:, 15:].min())
node_feature = all_data.iloc[:, 15:]
# print(node_feature)

In [ ]:
node_feature_norm = node_feature.to_numpy()
g.ndata['h'] = torch.from_numpy(node_feature_norm).float()
g.ndata['event'] = events
g.ndata['label'] = labels
g.ndata
# g.adj = adj
# add edges
src, dst = tuple(zip(*edge_list))
g.add_edges(src, dst)
# add edge weight
edge_wight = np.array(edge_wight)
g.edata['w'] = torch.from_numpy(edge_wight).float()

In [ ]:
class SAGE(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats, dropout=0, activation = None,aggregator_type='mean'):
        super().__init__()
        self.fc1 = nn.Linear(in_feats, hid_feats) 
        self.conv1 = SAGEConv(in_feats=hid_feats, out_feats=64, aggregator_type=aggregator_type, activation=activation, feat_drop=dropout)
        self.conv2 = SAGEConv(in_feats=64, out_feats= out_feats, aggregator_type=aggregator_type, activation=activation, feat_drop=dropout)
        self.fc2 = nn.Linear(out_feats, 1) 
    def forward(self, graph, inputs, w_input):
        # inputs are features of nodes
#         graph.ndata['h']= self.fc1(inputs)
        h = self.fc1(inputs)
        if w_input != None:
            h = self.conv1(graph, h, w_input)
        else:
            h = self.conv1(graph, h)
        h = self.conv2(graph,h)
#         print(h.size())
#         output=F.relu(self.fc2(h))
        output=self.fc2(h)

        return output

In [ ]:
class GCN(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats, activation = F.softmax, norm ="both"):
        super().__init__()
        self.fc1 = nn.Linear(in_feats, hid_feats) 
        self.conv1 = GraphConv(in_feats=hid_feats, out_feats= hid_feats, activation=activation, norm=norm)
        self.conv2 = GraphConv(in_feats=hid_feats, out_feats= out_feats,  activation=activation, norm=norm)
        self.fc2 = nn.Linear(out_feats, 1) 
        
    def forward(self, graph, inputs, w_input):
        # inputs are features of nodes
        h= self.fc1(inputs)
        h = self.conv1(graph, h)
        h = self.conv2(graph,h)
#         output=F.relu(self.fc2(h))
        h=self.fc2(h)
        
        return h

In [ ]:
def train(g, model, save_dic, idx_train,idx_val, idx_test, total_epoch=100, patience=5, lr=0.001, reg_l2=0, weight_decay=0.0001):
    model_name = save_dic['model']+str(save_dic['hid_feats'])+str(save_dic['out_feats'])+str(save_dic['reg_l2'])+save_dic["aggregator_type"]
    optimizer = torch.optim.Adam(model.parameters(),lr=lr, weight_decay=weight_decay)
    best_cindex = 0
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',factor=0.5, patience=patience, min_lr = 0.0001, verbose=True)
    criterion = NegativeLogLikelihood(reg_l2)
    features = g.ndata['h']
#     e_feature = g.edata['w']
    e_feature = None
    labels = g.ndata['label']
    events = g.ndata['event']
    t_total = time.time()
    with tqdm(range(total_epoch)) as t:
        for epoch in t:
            t.set_description('Epoch %d' % epoch)
            start = time.time()
            model.train()
            optimizer.zero_grad()
            output = model(g, features,e_feature)
            # Compute loss
            # Note that you should only compute the losses of the nodes in the training set.
            loss_train = criterion(output[idx_train], labels[idx_train],events[idx_train], model).clone()
            auc_train = c_index(-output[idx_train], labels[idx_train],events[idx_train])
            
            loss_train.backward(retain_graph=True)
            optimizer.step()
            
            model.eval()
            val_output = model(g, features,e_feature)
            loss_val = criterion(val_output[idx_val], labels[idx_val],events[idx_val], model).clone()
            scheduler.step(loss_val)
            
            auc_val = c_index(-val_output[idx_val], labels[idx_val],events[idx_val])
            auc_test = c_index(-val_output[idx_test], labels[idx_test],events[idx_test])
            t.set_postfix(
                  {"train_loss":loss_train.item(), "val_loss":loss_val.item(),
                  "train_cindex":auc_train.item(), "val_auc":auc_val.item(),
                "lr":optimizer.param_groups[0]['lr']}) 